# Design new scenarios (benchmarks)
In order to design new benchmarks in the context of power networks, a configuration file should be created from which the benchmark is initiated. The LIPS platform via `ConfigManager` class ease this operation, and this notebook shows how to use its different functionalities.

The figure below presents the scheme of benchmarking platform which is composed of three distinct parts : 
* DataSet: Generate some dataset for training and evaluation 
* Benchmark: coordinates between different parts and allows train and evaluate an agumented simulator 
* Evaluation: once the benchmark done, it allow to evaluate the performance with respect to various point of views

![title](img/Benchmarking_scheme_v2.png)

In this notebook we concentrate on the middle module where a scenario should be defined and configured, and for which we could generate some data in `Notebook 01`, and on the basis of which we can evalute some baseline methods (see `Notebook 02`). 

## TOC:
* [Read an existing config](#first-bullet)
* [Create a new Benchmark](#second-bullet)
* [Create a new configuration file](#create-new)

_for the moment, I return to the parent directory to be able to access the lips modules, this cell is not necessary if the lips package is installed in future_

In [ ]:
import pathlib
from pprint import pprint
from lips.config import ConfigManager

## Read an existing config <a class="anchor" id="first-bullet"></a>

Create an object of `ConfigManager`class. By indicating an existing benchmark name, its options will be restored by the config manager. You can also indicate a path to load an existing configuration file or to store a new configuration file.

Here, for demonstration purpose we select a path to an existing configuration file for power grid use case. 

In [ ]:
LIPS_PATH = pathlib.Path().resolve().parent
CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"

Instantiate a config using the indicated configuration path.

In [ ]:
cm_bench1 = ConfigManager(section_name="Benchmark1",
                          path=CONFIG_PATH
                         )

We can see the list of available benchmarks (aka config sections).

In [ ]:
cm_bench1.config.sections()

### Analyze the options available for a selected benchmark

As we have selected `Benchmark1` as the section name, its options could be accessed easily using provided functions. We can print all the parameters set for this benchmark:

In [ ]:
print(cm_bench1)

#### Access the option values
One can also access the options using `get_option` function and by indicating the desired option key.

Set input features for `Benchmark1` of power grid use case. In power grid use case, in addition to injections (`prod_p`, `prod_v`, `load_p`, `load_q`), we consider also two supplementary features refering to the grid topology and the connectivity of the power lines (`topo_vect`, `line_status`). 

These supplementary variables are used as input for simple augmented simulators as `FullyConnected` neural network, however, they intervene in latent space in more sophisticated architectures as `LeapNet`. More from these architecture are provided in next notebooks.

In [ ]:
cm_bench1.get_option("attr_x") + cm_bench1.get_option("attr_tau")

Set of outputs `Benchmark1` of power grid use case.

In [ ]:
cm_bench1.get_option("attr_y")

Set of required evaluation criteria for `Benchmark1` of power grid use case. As we can observe, there are 4 different categories of evaluation criteria which are defined in our proposed benchmarking pipeline, which are:

- `ML`: Machine learning related metrics (computing the accuracy of augmented simulators);
- `Physics`: physics compliances which verify the physics laws (equations) on the predictions of an augmented simulator;
- `IndRed`: Industrial Readiness which verifies whether the proposed augmented simulators could be exploited in industriy;
- `OOD`: which verifies the out-of-distribution generalization capacity of the augmented simulators.

These metrics could vary from benchmark to benchmark wrt to the complexity and outputs of each benchmark.

In [ ]:
cm_bench1.get_option("eval_dict")

Lets see the evaluation metrics for another benchmark (`Benchmark2`).

In [ ]:
cm_bench2 = ConfigManager(section_name="Benchmark2",
                          path=CONFIG_PATH
                         )

We can see that, this benchmark includes more output variables to be predicted.

In [ ]:
pprint(cm_bench2.get_option("attr_y")) 

We can see that there are more physics compliances that should be verified for this more complex benchmark, as it includes more outputs to predicted using an augmented simulator.

In [ ]:
pprint(cm_bench2.get_option("eval_dict"))

### Add/Edit the options
We can edit the existing options for the selected benchmark (aka config section).

In [ ]:
cm_bench1.edit_config_option('attr_y', "('a_or')")

In [ ]:
cm_bench1.get_option("attr_y")

### Write this modification to `config.ini` file

In [ ]:
cm_bench1._write_config()

### Remove an option and update the config file 
We can also remove an option that is not required anymore.

Lets add a new option first, using the same `edit_config_option` function.

In [ ]:
cm_bench1.edit_config_option('new_attr', "('theta_or')")

In [ ]:
print(cm_bench1)

Now, we can remove this newly created option using `remove_config_option` function.

In [ ]:
cm_bench1.remove_config_option(option="new_attr")

In [ ]:
print(cm_bench1)

## Create a new section for a new scenario <a class="anchor" id="second-bullet"></a>

We can also define a complete set of options for a new benchmark, if requried. It could be started by instantiating the `ConfigManager` class and giving a new name to the benchmark.

In [ ]:
cm = ConfigManager(section_name="Benchmark_new",
                   path=CONFIG_PATH
                  )

You can create the configuration by adding the required options as parameter. No restriction for the name of attributes at this step. 

**Nb :** However, these names will be used afterwards by `PowerGridBenchmark` class to parameterize the benchmark.

In [ ]:
cm.create_config(attr_y="('p_or', 'a_or')")

If you print now the configuration options, you may see some options that came from the `DEFAULT` section of the configuration file. If this is not the required behavior, you can skip the rest of this section and see the the [next](#create-new) section in which we create a configuration file from scratch.

In [ ]:
print(cm)

add another option

In [ ]:
description = "'this benchmark is intended to output power and current'"
cm.edit_config_option(option='description', value=description)

In [ ]:
cm.get_option("description")

and remove an undesired option

In [ ]:
cm.remove_config_option(option="description")

We can see that this benchmark is added among the existing benchmarks.

In [ ]:
cm.config.sections()

We can verify its options.

*NB*: the returned options comprise also `DEFAULT` section options.

In [ ]:
cm.get_options_dict().keys()

And its values.

In [ ]:
cm.get_option('attr_y')

And finally write it to the file.

In [ ]:
cm._write_config()

Remove unused or test sections and update `config.ini`

In [ ]:
cm.remove_section(section_name="Benchmark_new")
cm._write_config()

## Create a new configuration file <a class="anchor" id="create-new"></a>

In [ ]:
LIPS_PATH = pathlib.Path().resolve().parent
CONFIG_FILE = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "new_config_file.ini"

You should start by creating the corresponding file.

In [ ]:
try:
    open(CONFIG_FILE, 'a').close()
except OSError:
    print('Failed creating the file')
else:
    print('File created')

In [ ]:
cm = ConfigManager(section_name="MyBenchmark",
                   path=CONFIG_FILE
                  )

Create the configuration section in the corresponding file.

In [ ]:
cm.create_config()

Add a set of required options for this custom benchmark.

In [ ]:
cm.edit_config_option(option="attr_x", value='("prod_p", "prod_v", "load_p", "load_q")')
cm.edit_config_option(option="attr_y", value="('p_or', 'a_or')")

Print the structure of you configuration section.

In [ ]:
print(cm)

You can add more options if required and write it out.

In [ ]:
cm._write_config()